In [ ]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np

In [ ]:
# volviendo a la documentación, la primera url que nos pone en la documentación es para sacar una frase de un personaje aleatorio de los simpson
# para eso en la casilla en blanco que tenemos a la derecha nos muestra la url que deberemos usar. Por lo tanto, la copiamos y la metemos en una variable. La podemos llamar como queramos, en nuestro caso, la llamamos "url"
url = 'https://thesimpsonsquoteapi.glitch.me/quotes'

# hacemos la petición de los datos a la API
response = requests.get(url=url)

# pedimos a la API que nos diga si lo estamos haciendo bien con el método status_code
response.status_code

# le pedimos que nos diga el por qué
response.reason


In [ ]:
# definimos nuestro diccionario
variable = {'count': 3}

# añadimos a la petición el parámetro headers para añadir esa información que queríamos
response = requests.get('https://thesimpsonsquoteapi.glitch.me/quotes', params=variable)

# vemos si estamos haciendo bien la llamada:
response.status_code

- EXTRAYENDO DATOS DE API

In [ ]:
# hagamoslo para la primera de las url que vimos de los simpsons, donde nos devolvía una frase aleatoria de un personaje aleatorio

# establecemos la url a la que queremos hacer la llamada
url = 'https://thesimpsonsquoteapi.glitch.me/quotes'

# hacemos la llamada usando el método get
response = requests.get(url=url)

# almacenamos el resultado en una variable que se llama "frase_json" donde tendremos el json
frase_json = response.json()

# almacenamos el resultado en una variable que se llama "frase_str" donde tendremos el string
frase_str = response.text

In [ ]:
# vemoas otro ejemplo, en este caso con la llamada a la API con un número de frases concreto

# definimos nuestro diccionario
variable = {'count': "3"}

# añadimos a la petición el parámetro headers para añadir esa información que queríamos
response2 = requests.get(url=f'https://thesimpsonsquoteapi.glitch.me/quotes',params=variable)

# vemos si estamos haciendo bien la llamada:
response2.status_code


maggie_df = pd.json_normalize(response2.json())
maggie_df

- ALMACENANDO DATASET

In [ ]:
maggie_df.to_pickle('../files/API_simpsons.pkl')

- LIBRERIA DATETIME

ayer = datetime.now() - timedelta(1)
ayer

In [ ]:
# FORMATEO FECHA 
ayer = datetime.strftime(ayer, '%Y-%m-%d')
ayer

- FUNCION ACTUALIZACION DATOS 

In [ ]:
def llamada_API(lat, lon, producto):
    
    # hacemos la llamada  a la API
    url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
    if codigo_estado == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

    # convertimos los resultados en un dataframe: 
    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    return df

- FUNCION LIMPIAR/AÑADIR DF NUEVOS DATOS 

In [ ]:
def limpiar_civil(df_nuevo, lat, long, ciudad): 
    
    # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
    # En este caso será la fecha del día de hoy. 
    hoy = datetime.now()
    hoy = datetime.strftime(hoy, '%Y-%m-%d')
    
    # creamos la nueva columna
    df_nuevo["fecha"] = hoy
    
    df_nuevo["latitud"] = lat
    df_nuevo["longitud"] = long
    df_nuevo["ciudad"] = ciudad
    return df_nuevo

- AÑADIENDO OTROS DATOS DE LA API

In [ ]:
# la latitud y la longitud serán las mismas
latitud = 40.4165
longitud = -3.70256

# pero en este caso el producto cambiará. En este caso "astro"
producto2 = "astro"

In [ ]:
# llamamamos a la función 

df_astro = llamada_API(latitud, longitud, producto2)

In [ ]:
def limpiar_astro(df, lat, long, ciudad):
    
    #seleccionamos solo las columnas que nos interesan
    df = df[["seeing", "transparency", "timepoint"]]
    
    # creamos la columna de fecha: 
    hoy = datetime.now()
    hoy = datetime.strftime(hoy, '%Y-%m-%d')
    df["fecha"] = hoy
    
    
    # insertamos las columnas de la localidad
    df["ciudad"] = ciudad
    
    return df

- FUNCION PARA UNIR DF, METODO DEPENDE (CONCAT, MERGE, JOIN)

In [ ]:
def juntar_dfs(df_completo, df_visibilidad):
    
    # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
    # en este caso el how lo ponemos como right ya que queremos que se quede con toda la info que en la segunda tabla que le pasamos que es la que tiene toda la información
    df = pd.merge(df_completo , df_astro , on=['fecha', "timepoint", "ciudad"], how = "right")

    
    return df

- FUNCION CHEQUEANDO LOS DATOS 

In [ ]:
def chequear_datos(df): 
    
    print("Las columnas son:", "\n")
    print(list(df.columns))
    print("-----------------------------------------")
    
    print("Los tipos de datos que tenemos son:", "\n")
    print(df.dtypes)
    print("-----------------------------------------")
    
    print("El porcentaje de nulos:", "\n")
    print((df.isnull().sum() / df.shape[0]) *  100)

- FUNCION LIMPIANDO Y GUARDANDO 

In [ ]:
def limpiar_dataframe(df, lista_columnas = []): 
    
    #convertimos la fecha a datetime
    df["fecha"] = pd.to_datetime(df["fecha"])
    
    # reemplazamos los nulos de las columnas por la media
     # lista de columnas en las que queremos reemplazar los nulos
    if len(lista_columnas) != 0:
        df[lista_columnas]=df[lista_columnas].fillna(df.median())
    else:
        pass
    
    
    # quitar % 
    df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)
    
    # guardamos los datos una vez limpios
    df.to_pickle('../files/datos_actualizados.pkl')
    df.to_csv('../files/datos_actualizados.csv')
    
    return df